# Основни пример за AutoGen

У овом примеру кода, користићете [AutoGen](https://aka.ms/ai-agents/autogen) AI оквир за креирање основног агента.

Циљ овог примера је да вам покаже кораке које ћемо касније користити у додатним примерима кода приликом имплементације различитих агентичких образаца.


## Увези потребне Python пакете


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Креирајте клијента

У овом примеру, користићемо [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) за приступ LLM-у.

`model` је дефинисан као `gpt-4o-mini`. Пробајте да промените модел на неки други доступан у GitHub Models продавници како бисте видели различите резултате.

Као брзи тест, само ћемо покренути једноставан упит - `Који је главни град Француске`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Дефинисање агента

Сада када смо подесили `client` и потврдили да ради, хајде да креирамо `AssistantAgent`. Сваком агенту могу се доделити:
**име** - Кратко име које ће бити корисно за референцирање у токовима са више агената.
**model_client** - Клијент који сте креирали у претходном кораку.
**tools** - Доступни алати које агент може користити за обављање задатка.
**system_message** - Метапромпт који дефинише задатак, понашање и тон LLM-а.

Можете променити системску поруку да бисте видели како LLM реагује. О алатима ћемо говорити у лекцији #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Покрени агента

Следећа функција ће покренути агента. Користимо метод `on_message` да ажурирамо стање агента новом поруком.

У овом случају, ажурирамо стање новом поруком од корисника која гласи: `"Plan me a great sunny vacation"`.

Можете променити садржај поруке да бисте видели како LLM различито реагује.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
